In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


choose the epoch data with losses close to zero <br> From careful inspection I would choose the data from epochs 89,91,97,100<br>
stored in the location
'/content/drive/MyDrive/Thesis data/recheckgen4/Generated_ep{epoch}'




In [ ]:
# #Run this block when starting colab for first time , this is used to install auto sklearn for classification.
# !sudo apt-get install build-essential swig
# !pip install auto-sklearn==0.10.0

In [ ]:
import numpy as np
import pickle as pickle
import pandas as pd
from scipy.stats import skew,kurtosis
import matplotlib.pyplot as plt



from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold,RepeatedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics import classification_report,confusion_matrix,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate,cross_val_score
from sklearn.utils import shuffle

import os
import tensorflow as tf
import time
import sys
sys.path.append("..")
# import autosklearn.classification

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPool2D, Flatten, Dense, Dropout,Activation,MaxPooling2D,ReLU,Add,GlobalAvgPool2D
from tensorflow.keras import Model,Sequential
from tensorflow.keras.activations import relu
from tensorflow.keras.optimizers import SGD,Adam,RMSprop
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
A = np.load('/content/drive/MyDrive/Thesis data/A.npy') # File with all(40) channel data
B = np.load('/content/drive/MyDrive/Thesis data/B.npy') #File with only data from left hemisphere 
A_norm = np.load('/content/drive/MyDrive/Thesis data/Anorm.npy') # normalised 40 channel data
B_norm = np.load('/content/drive/MyDrive/Thesis data/Anorm.npy') # normalised 7 channel data
print("A.shape =",A.shape)
print("B.shape =",B.shape)

#Load the labels now    
valence = np.genfromtxt('/content/drive/MyDrive/Thesis data/label_class_0.dat', delimiter=' ') # for all 32 subjects
arousal = np.genfromtxt('/content/drive/MyDrive/Thesis data/label_class_1.dat', delimiter=' ') # for all 32 subjects
dominance = np.genfromtxt('/content/drive/MyDrive/Thesis data/label_class_2.dat', delimiter=' ') # for all 32 subjects
liking = np.genfromtxt('/content/drive/MyDrive/Thesis data/label_class_3.dat', delimiter=' ') # for all 32 subjects
 

A.shape = (51200, 99)
B.shape = (8960, 99)


## Import the train and test data

In [ ]:
# traindata
traindata = np.load('/content/drive/MyDrive/Thesis data/traindata22.npy') # training data for baseline and also the data for appending to the model

testdata = np.load('/content/drive/MyDrive/Thesis data/testdata10.npy') # testdata for baseline and all of the augmented models




#this is the training data labels
val22 = np.load('/content/drive/MyDrive/Thesis data/val22.npy') #22 subjects of data = 880 experiments
aro22 = np.load('/content/drive/MyDrive/Thesis data/aro22.npy') #22 subjects of data = 880 experiments
dom22 = np.load('/content/drive/MyDrive/Thesis data/dom22.npy') #22 subjects of data = 880 experiments
lik22 = np.load('/content/drive/MyDrive/Thesis data/lik22.npy') #22 subjects of data = 880 experiments


# this is the test data labels
val10 = np.load('/content/drive/MyDrive/Thesis data/val10.npy') #10 subjects of data = 400 experiments
aro10 = np.load('/content/drive/MyDrive/Thesis data/aro10.npy') #10 subjects of data = 400 experiments
dom10 = np.load('/content/drive/MyDrive/Thesis data/dom10.npy') #10 subjects of data = 400 experiments
lik10 = np.load('/content/drive/MyDrive/Thesis data/lik10.npy') #10 subjects of data = 400 experiments




## Augmenting data below
Start with the labels<br>
For example yva2 means valence augmented twice<br>
ar3 means arousal augmented thrice

In [ ]:
###################################
# Augmenting the labels 
# 2x
yva2 = np.hstack((val22,val22))
yar2 = np.hstack((aro22,aro22))
ydo2 = np.hstack((dom22,dom22))
yli2 = np.hstack((lik22,lik22))


# 3x
yva3 = np.hstack((val22,val22,val22))
yar3 = np.hstack((aro22,aro22,aro22))
ydo3 = np.hstack((dom22,dom22,dom22))
yli3 = np.hstack((lik22,lik22,lik22))

# 4x
yva4 = np.hstack((val22,val22,val22,val22))
yar4 = np.hstack((aro22,aro22,aro22,aro22))
ydo4 = np.hstack((dom22,dom22,dom22,dom22))
yli5 = np.hstack((lik22,lik22,lik22,lik22))


# 5x
yva5 = np.hstack((val22,val22,val22,val22,val22))
yar5 = np.hstack((aro22,aro22,aro22,aro22,aro22))
ydo5 = np.hstack((dom22,dom22,dom22,dom22,dom22))
yli5 = np.hstack((lik22,lik22,lik22,lik22,lik22))



## Get some generated data points<br>
Get the data points with generator and discriminator close to zero and least as possible

From submitWGAN-GP.ipynb it is noted that the data from epochs 89, 91, 97, 100 will be chosen

In [ ]:
g1 =np.load('/content/drive/MyDrive/Thesis data/3rdGAN/Generated_ep89.npy') # generated data with all(40) channel data

g2 =np.load('/content/drive/MyDrive/Thesis data/3rdGAN/Generated_ep91.npy') # generated data with all(40) channel data

g3 =np.load('/content/drive/MyDrive/Thesis data/3rdGAN/Generated_ep97.npy') # generated data with all(40) channel data

g4 =np.load('/content/drive/MyDrive/Thesis data/3rdGAN/Generated_ep100.npy') # generated data with all(40) channel data




g1.shape

(880, 40, 99)

## Augment the data

In [ ]:
################################### 

# Augmenting the data  (add the original training data to the generated data)
print("Augmenting data............")

# 2x
x2 = np.concatenate([g4, traindata], 0) 
print(x2.shape)

# 3x
x3 = np.concatenate([g1,g2, traindata], 0)
print(x3.shape)

# 4x
x4 = np.concatenate([g1,g2,g3 ,traindata], 0)
print(x4.shape)


# 5x
x5 = np.concatenate([g1,g2,g3,g4, traindata], 0)
print(x5.shape)



Augmenting data............
(1760, 40, 99)
(2640, 40, 99)
(3520, 40, 99)
(4400, 40, 99)


# KNN

In [ ]:

print("Augmented data x2 shape:",x2.shape)
print(yva2.shape,yar2.shape,ydo2.shape,yli2.shape)


Augmented data x2 shape: (1760, 40, 99)
(1760,) (1760,) (1760,) (1760,)


In [ ]:
# Define KNN based model for actual/original dataset
def knnmodel(xtrain,ytrain,xtest,ytest,kval):
  random_state=100
  # print("xtrain.shape,ytrain.shape",xtrain.shape,ytrain.shape)
  # print("xtest.shape,ytest.shape",xtrain.shape,ytrain.shape)
  # Define the KNN model here

  model = KNeighborsClassifier(n_neighbors=kval) 
  model.fit(xtrain, ytrain)
  y_pred = model.predict(xtest)

  # print("Accuracy score for fold",fold_no)
  acc = accuracy_score(ytest, y_pred)*100
  acc = round(acc, 4)
  print("Accuracy",acc)
  # print(classification_report(ytest, y_pred))
  # print(confusion_matrix(ytest,y_pred))
  # return acc,xtrain, xtest, ytrain, ytest
  return acc

In [ ]:
# Define KNN based model for augmented data
def knnmodel2(xtrain,ytrain,xtest,ytest,kval):
  
  # print("xtrain.shape,ytrain.shape",xtrain.shape,ytrain.shape)
  # print("xtest.shape,ytest.shape",xtest.shape,ytest.shape)

  # Define the KNN model here

  model = KNeighborsClassifier(n_neighbors=kval)
  model.fit(xtrain, ytrain)
  y_pred = model.predict(xtest)

  # print("Accuracy score for fold",fold_no)
  acc = accuracy_score(ytest, y_pred)*100
  acc = round(acc, 4)
  print("Accuracy",acc)
  print(classification_report(ytest, y_pred))
  print(confusion_matrix(ytest,y_pred))
  # return acc,xtrain, xtest, ytrain, ytest
  return acc

## Using 2X data

In [ ]:
# Using 2x data

trainlabel_list = [yva2,yar2,ydo2,yli2]
testlabel_list = [val10,aro10,dom10,lik10]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []



for k in range(1,50):
  print("K value =",k)
  for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
    print("#########################################",name,"#########################################")
    sc = knnmodel(x2.reshape(-1, 40*99),traain,testdata.reshape(-1, 40*99),tesst,kval = k)
    scorelist.append(sc)
  


# for i,name in zip(scorelist,names):
#   print ("Accuracy of ",name ,i)

######################################### Valence #########################################
Accuracy 56.5
######################################### Arousal #########################################
Accuracy 53.5
######################################### Dominance #########################################
Accuracy 56.5
######################################### Liking #########################################
Accuracy 56.5
K value = 1
######################################### Valence #########################################
Accuracy 52.25
######################################### Arousal #########################################
Accuracy 47.0
######################################### Dominance #########################################
Accuracy 53.75
######################################### Liking #########################################
Accuracy 44.5
K value = 2
######################################### Valence #########################################
Accuracy 54.75
#################

Best results - 
Accuracy of  Valence 61.75 <br>
Accuracy of  Arousal 57 <br>
Accuracy of  Dominance 61.5  <br>
Accuracy of  Liking 71.5 <br>


## Using 3x data

In [ ]:
# Using 3x data

trainlabel_list = [yva3,yar3,ydo3,yli3]
testlabel_list = [val10,aro10,dom10,lik10]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []



for k in range(1,50):
  print("K value =",k)
  for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
    print("#########################################",name,"#########################################")
    sc = knnmodel(x3.reshape(-1, 40*99),traain,testdata.reshape(-1, 40*99),tesst,kval = k)
    scorelist.append(sc)
  


K value = 1
######################################### Valence #########################################
Accuracy 57.75
######################################### Arousal #########################################
Accuracy 53.25
######################################### Dominance #########################################
Accuracy 56.25
######################################### Liking #########################################
Accuracy 57.75
K value = 2
######################################### Valence #########################################
Accuracy 51.5
######################################### Arousal #########################################
Accuracy 46.75
######################################### Dominance #########################################
Accuracy 53.0
######################################### Liking #########################################
Accuracy 46.5
K value = 3
######################################### Valence #########################################
Accuracy 56.0
###

Best results <br>
Accuracy of  Valence 62 <br>
Accuracy of  Arousal 59 <br>
Accuracy of  Dominance 61.25 <br>
Accuracy of  Liking 70.25 <br>


## Using 5X data

In [ ]:
# Using 5x data

trainlabel_list = [yva5,yar5,ydo5,yli5]
testlabel_list = [val10,aro10,dom10,lik10]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []



for k in range(1,50):
  print("\n")
  print("K value =",k)
  for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
    print("#########################################",name,"#########################################")
    sc = knnmodel(x5.reshape(-1, 40*99),traain,testdata.reshape(-1, 40*99),tesst,kval = k)
    scorelist.append(sc)
  




K value = 1
######################################### Valence #########################################
Accuracy 56.25
######################################### Arousal #########################################
Accuracy 53.5
######################################### Dominance #########################################
Accuracy 55.75
######################################### Liking #########################################
Accuracy 57.0


K value = 2
######################################### Valence #########################################
Accuracy 51.5
######################################### Arousal #########################################
Accuracy 47.25
######################################### Dominance #########################################
Accuracy 53.0
######################################### Liking #########################################
Accuracy 46.5


K value = 3
######################################### Valence #########################################
Accuracy 55.5

Best results <br>
Accuracy of  Valence 62.5 <br>
Accuracy of  Arousal 59.25 <br>
Accuracy of  Dominance 61 <br>
Accuracy of  Liking 70.5 <br>


# SVM

In [ ]:
# # Define SVM based model for actual/original dataset
# def svmmodel(xtrain,ytrain,xtest,ytest,c,kernel = 'linear'):

#   # SVM

#   model = SVC(C=c,kernel=kernel,gamma = 'auto')
#   model.fit(xtrain, ytrain)
#   y_pred = model.predict(xtest)

#   # print("Accuracy score for fold",fold_no)
#   acc = accuracy_score(ytest, y_pred)*100
#   acc = round(acc, 4)
#   print("Accuracy",acc)
#   # print(classification_report(ytest, y_pred))
#   # print(confusion_matrix(ytest,y_pred))
#   # return acc,xtrain, xtest, ytrain, ytest
#   return acc

In [ ]:
# Define SVM based model for augmented data
def svmmodel2(xtrain,ytrain,xtest,ytest,c,kernel = 'linear'):

  # SVM model

  model = SVC(C=c,kernel=kernel,gamma = 'auto')
  model.fit(xtrain, ytrain)
  y_pred = model.predict(xtest)

  # print("Accuracy score for fold",fold_no)
  acc = accuracy_score(ytest, y_pred)*100
  acc = round(acc, 4)
  print("Accuracy",acc)
  # print(classification_report(ytest, y_pred))
  # print(confusion_matrix(ytest,y_pred))
  # return acc,xtrain, xtest, ytrain, ytest
  return acc

In [ ]:
# Using actual data
trainlabel_list = [val22,aro22,dom22,lik22]
testlabel_list = [val10,aro10,dom10,lik10]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
cval = [0.0001,0.001,0.01,0.1,1,10,100]

for c in cval:
  print("\n")
  print("C value =",c)
  for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
    print("#########################################",name,"#########################################")
    sc = svmmodel(g4.reshape(-1, 40*99),traain,testdata.reshape(-1, 40*99),tesst,c = c)
    scorelist.append(sc)






C value = 0.0001
######################################### Valence #########################################
Accuracy 57.75
######################################### Arousal #########################################
Accuracy 55.75
######################################### Dominance #########################################
Accuracy 57.25
######################################### Liking #########################################
Accuracy 70.5


C value = 0.001
######################################### Valence #########################################
Accuracy 57.75
######################################### Arousal #########################################
Accuracy 55.75
######################################### Dominance #########################################
Accuracy 57.25
######################################### Liking #########################################
Accuracy 70.5


C value = 0.01
######################################### Valence ########################################

## Using 2x data

In [ ]:
# Using 2x data

trainlabel_list = [yva2,yar2,ydo2,yli2]
testlabel_list = [val10,aro10,dom10,lik10]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
cval = [0.00001,0.0001,0.001,0.01,0.1,1,10,100]

for c in cval:
  print("\n")
  print("C value =",c)
  for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
    print("#########################################",name,"#########################################")
    sc = svmmodel(x2.reshape(-1, 40*99),traain,testdata.reshape(-1, 40*99),tesst,c = c)
    scorelist.append(sc)






C value = 1e-05
######################################### Valence #########################################
Accuracy 57.75
######################################### Arousal #########################################
Accuracy 55.75
######################################### Dominance #########################################
Accuracy 57.25
######################################### Liking #########################################
Accuracy 70.5


C value = 0.0001
######################################### Valence #########################################
Accuracy 57.75
######################################### Arousal #########################################
Accuracy 55.75
######################################### Dominance #########################################
Accuracy 57.25
######################################### Liking #########################################
Accuracy 70.5


C value = 0.001
######################################### Valence #######################################

Best results <br>
Accuracy of  Valence 60.25 <br>
Accuracy of  Arousal 55.75 <br>
Accuracy of  Dominance 59.5 <br>
Accuracy of  Liking 70.5 <br>


## Using 3x data

In [ ]:
# Using 3x data

trainlabel_list = [yva3,yar3,ydo3,yli3]
testlabel_list = [val10,aro10,dom10,lik10]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
cval = [0.00001,0.0001,0.001,0.01,0.1,1,10,100]

for c in cval:
  print("\n")
  print("C value =",c)
  for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
    print("#########################################",name,"#########################################")
    sc = svmmodel(x3.reshape(-1, 40*99),traain,testdata.reshape(-1, 40*99),tesst,c = c)
    scorelist.append(sc)






C value = 1e-05
######################################### Valence #########################################
Accuracy 57.75
######################################### Arousal #########################################
Accuracy 55.75
######################################### Dominance #########################################
Accuracy 57.25
######################################### Liking #########################################
Accuracy 70.5


C value = 0.0001
######################################### Valence #########################################
Accuracy 57.75
######################################### Arousal #########################################
Accuracy 55.75
######################################### Dominance #########################################
Accuracy 57.25
######################################### Liking #########################################
Accuracy 70.5


C value = 0.001
######################################### Valence #######################################

Best results <br>
Accuracy of  Valence 60.5 <br>
Accuracy of  Arousal 57.5 <br>
Accuracy of  Dominance 59.75 <br>
Accuracy of  Liking 70.5 <br>


## Using 5x data

In [ ]:
# Using 5x data
trainlabel_list = [yva5,yar5,ydo5,yli5]
testlabel_list = [val10,aro10,dom10,lik10]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
cval = [0.00001,0.0001,0.001,0.01,0.1,1,10,100]

for c in cval:
  print("\n")
  print("C value =",c)
  for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
    print("#########################################",name,"#########################################")
    sc = svmmodel(x5.reshape(-1, 40*99),traain,testdata.reshape(-1, 40*99),tesst,c = c)
    scorelist.append(sc)






C value = 1e-05
######################################### Valence #########################################
Accuracy 57.75
######################################### Arousal #########################################
Accuracy 55.75
######################################### Dominance #########################################
Accuracy 57.25
######################################### Liking #########################################
Accuracy 70.5


C value = 0.0001
######################################### Valence #########################################
Accuracy 57.75
######################################### Arousal #########################################
Accuracy 55.75
######################################### Dominance #########################################
Accuracy 57.25
######################################### Liking #########################################
Accuracy 70.5


C value = 0.001
######################################### Valence #######################################

Best results <br>
Accuracy of  Valence 59.75 <br>
Accuracy of  Arousal 57 <br>
Accuracy of  Dominance 58.25 <br>
Accuracy of  Liking 70.5 <br>


# Try neural networks

In [ ]:
# Split the data into training/testing sets
def deepmodel(modelnum,xtrain,ytrain,xtest,ytest,nepochs,batch_size):
  # ()
  
  ytrain = to_categorical(ytrain)
  ytest = to_categorical(ytest)



  input_shape = (40, 99, 1)
  # print(xtrain.shape,xtest.shape)
  xtrain = np.reshape(xtrain,[-1, 40, 99, 1])
  xtest = np.reshape(xtest,[-1, 40, 99, 1])
  print("xtrain.shape,ytrain.shape")
  print(xtrain.shape,ytrain.shape)
  if modelnum == 1:
  # Model 1
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    # model.add(Dense(1, activation='sigmoid'))
  elif modelnum == 2:
    # Resnet kind of model

    def conv_bn_relu(x, filters, kernel_size, strides):
        x = Conv2D(filters=filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same')(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        return x


    def identity_block(tensor, filters):
        x = conv_bn_relu(tensor, filters=filters, kernel_size=1, strides=1)
        x = conv_bn_relu(x, filters=filters, kernel_size=3, strides=1)
        x = Conv2D(filters=4*filters, kernel_size=1, strides=1)(x)  # notice: filters=4*filters
        x = BatchNormalization()(x)

        x = Add()([x, tensor])
        x = ReLU()(x)
        return x


    def projection_block(tensor, filters, strides):
        # left branch
        x = conv_bn_relu(tensor, filters=filters, kernel_size=1, strides=1)
        x = conv_bn_relu(x, filters=filters, kernel_size=3, strides=strides)
        x = Conv2D(filters=4*filters, kernel_size=1, strides=1)(x)  # notice: filters=4*filters
        x = BatchNormalization()(x)

        # right branch
        shortcut = Conv2D(filters=4*filters, kernel_size=1, strides=strides)(tensor)  # notice: filters=4*filters
        shortcut = BatchNormalization()(shortcut)

        x = Add()([x, shortcut])
        x = ReLU()(x)
        return x


    def resnet_block(x, filters, reps, strides):
        x = projection_block(x, filters=filters, strides=strides)
        for _ in range(reps-1):  # the -1 is because the first block was a Conv one
            x = identity_block(x, filters=filters)
        return x


    input = Input(shape=(40, 99, 1))

    x = conv_bn_relu(input, filters=64, kernel_size=7, strides=2)  # [3]: 7x7, 64, strides 2
    x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)  # [3]: 3x3 max mool, strides 2

    x = resnet_block(x, filters=64, reps=3, strides=1)
    x = resnet_block(x, filters=128, reps=4, strides=2)  # s=2 ([2]: conv3_1)
    x = resnet_block(x, filters=256, reps=6, strides=2)  # s=2 ([2]: conv4_1)
    x = resnet_block(x, filters=512, reps=3, strides=2)  # s=2 ([2]: conv5_1)

    x = GlobalAvgPool2D()(x)  # [3]: average pool *it is not written any pool size so we use Global

    # # output = Dense(1, activation='sigmoid')(x)
    output = Dense(2, activation='softmax')(x)
    model = Model(input, output)



  model.compile(loss= 'categorical_crossentropy',#'binary_crossentropy','categorical_crossentropy'
                    optimizer=Adam(),
                    metrics=['accuracy'])
  # Fit data to model
  history = model.fit(xtrain,ytrain ,
            batch_size = batch_size,
            epochs=nepochs,
            verbose=1)

  # Generate generalization metrics
  scores = model.evaluate(xtest, ytest, verbose=0)#verbose =1 
  y_pred = model.predict(xtest)
  #     print("Fold number",fold_no)
  acc = scores[1]*100
  acc = round(acc, 4)
  print("Accuracy",acc)
  print("Loss",scores[0])
  # print(classification_report(ytest.argmax(axis=1), y_pred.argmax(axis=1)))
  # print(confusion_matrix(ytest.argmax(axis=1), y_pred.argmax(axis=1)))
  return acc

## Using 2x data

In [ ]:
# Using 2x data
trainlabel_list = [yva2,yar2,ydo2,yli2]
testlabel_list = [val10,aro10,dom10,lik10]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
eps = 5 # epochs
bs = 32 # batch size
div = 2
# modelnum,xtrain,ytrain,xtest,ytest,nepochs,batch_size

for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
  print("#########################################",name,"#########################################")
  sc = deepmodel(1,x2.reshape(-1, 40,99),traain,testdata.reshape(-1, 40,99),tesst,nepochs=eps,batch_size=bs)
  scorelist.append(sc)




######################################### Valence #########################################
xtrain.shape,ytrain.shape
(1760, 40, 99, 1) (1760, 2)
Epoch 1/5
55/55 [==============================] - 1s 5ms/step - loss: 0.6942 - accuracy: 0.5314
Epoch 2/5
55/55 [==============================] - 0s 5ms/step - loss: 0.6878 - accuracy: 0.5470
Epoch 3/5
55/55 [==============================] - 0s 5ms/step - loss: 0.6884 - accuracy: 0.5489
Epoch 4/5
55/55 [==============================] - 0s 5ms/step - loss: 0.6805 - accuracy: 0.5666
Epoch 5/5
55/55 [==============================] - 0s 5ms/step - loss: 0.6873 - accuracy: 0.5463
Accuracy 58.75
Loss 0.6759659647941589
######################################### Arousal #########################################
xtrain.shape,ytrain.shape
(1760, 40, 99, 1) (1760, 2)
Epoch 1/5
55/55 [==============================] - 1s 5ms/step - loss: 0.7163 - accuracy: 0.5432
Epoch 2/5
55/55 [==============================] - 0s 5ms/step - loss: 0.6833 - accurac

Accuracy of  Valence – 59.25 <br>
Accuracy of  Arousal 59 <br>
Accuracy of  Dominance 59.75 <br>
Accuracy of  Liking 71 <br>


## Using 3x data

In [ ]:
# Using 2x data
trainlabel_list = [yva3,yar3,ydo3,yli3]
testlabel_list = [val10,aro10,dom10,lik10]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
eps = 5 # epochs
bs = 32 # batch size
div = 2
# modelnum,xtrain,ytrain,xtest,ytest,nepochs,batch_size

for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
  print("#########################################",name,"#########################################")
  sc = deepmodel(1,x3.reshape(-1, 40,99),traain,testdata.reshape(-1, 40,99),tesst,nepochs=eps,batch_size=bs)
  scorelist.append(sc)




######################################### Valence #########################################
xtrain.shape,ytrain.shape
(2640, 40, 99, 1) (2640, 2)
Epoch 1/5
83/83 [==============================] - 1s 6ms/step - loss: 0.7415 - accuracy: 0.5314
Epoch 2/5
83/83 [==============================] - 0s 5ms/step - loss: 0.6891 - accuracy: 0.5488
Epoch 3/5
83/83 [==============================] - 0s 5ms/step - loss: 0.6914 - accuracy: 0.5380
Epoch 4/5
83/83 [==============================] - 0s 5ms/step - loss: 0.6906 - accuracy: 0.5362
Epoch 5/5
83/83 [==============================] - 0s 5ms/step - loss: 0.6873 - accuracy: 0.5456
Accuracy 58.0
Loss 0.6811321377754211
######################################### Arousal #########################################
xtrain.shape,ytrain.shape
(2640, 40, 99, 1) (2640, 2)
Epoch 1/5
83/83 [==============================] - 1s 5ms/step - loss: 0.6925 - accuracy: 0.5460
Epoch 2/5
83/83 [==============================] - 0s 5ms/step - loss: 0.6824 - accuracy

Best accuracies <br>
Accuracy of  Valence 61.5 <br>
Accuracy of  Arousal 59 <br>
Accuracy of  Dominance 59.5 <br>
Accuracy of  Liking 70.5 <br>


## Using 5x data

In [ ]:
# Using 2x data
trainlabel_list = [yva5,yar5,ydo5,yli5]
testlabel_list = [val10,aro10,dom10,lik10]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
eps = 5 # epochs
bs = 32 # batch size
div = 2
# modelnum,xtrain,ytrain,xtest,ytest,nepochs,batch_size

for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
  print("#########################################",name,"#########################################")
  sc = deepmodel(1,x5.reshape(-1, 40,99),traain,testdata.reshape(-1, 40,99),tesst,nepochs=eps,batch_size=bs)
  scorelist.append(sc)




######################################### Valence #########################################
xtrain.shape,ytrain.shape
(4400, 40, 99, 1) (4400, 2)
Epoch 1/5
138/138 [==============================] - 1s 6ms/step - loss: 0.7078 - accuracy: 0.5080
Epoch 2/5
138/138 [==============================] - 1s 5ms/step - loss: 0.6894 - accuracy: 0.5446
Epoch 3/5
138/138 [==============================] - 1s 5ms/step - loss: 0.6899 - accuracy: 0.5413
Epoch 4/5
138/138 [==============================] - 1s 5ms/step - loss: 0.6889 - accuracy: 0.5431
Epoch 5/5
138/138 [==============================] - 1s 5ms/step - loss: 0.6890 - accuracy: 0.5387
Accuracy 57.75
Loss 0.6826289296150208
######################################### Arousal #########################################
xtrain.shape,ytrain.shape
(4400, 40, 99, 1) (4400, 2)
Epoch 1/5
138/138 [==============================] - 1s 5ms/step - loss: 0.7044 - accuracy: 0.5523
Epoch 2/5
138/138 [==============================] - 1s 5ms/step - loss: 0.

Best values <br>
Accuracy of  Valence 58.25 <br>
Accuracy of  Arousal 57.75 <br>
Accuracy of  Dominance 59.25 <br>
Accuracy of  Liking 70.5 <br>



# Try keras tuner

Keras tuner is state of the art for finding the right hyper-parameters

In [ ]:
!pip install -q -U keras-tuner
import kerastuner as kt
import IPython

     |████████████████████████████████| 71kB 5.4MB/s 


In [ ]:
# Try keras tuner
def model_builder(hp):
  model = Sequential()
  model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(40, 99,1)))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Flatten())
  
  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(Dense(units = hp_units, activation = 'relu'))
  model.add(Dense(2, activation='softmax'))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
  model.compile(loss= 'categorical_crossentropy',#'binary_crossentropy','categorical_crossentropy'
                    optimizer=Adam(learning_rate = hp_learning_rate),
                    metrics=['accuracy'])
  # model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
  #               loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
  #               metrics = ['accuracy'])

  return model

class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [ ]:
tuner = kt.Hyperband(model_builder,overwrite = True,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3
                     )

In [ ]:
# xtrain, xtest, ytrain, ytest = train_test_split(A_norm.reshape(-1, 40 , 99), to_categorical(liking), test_size=testsize ,shuffle = True,random_state=100)
#########################################
## for augmented models
random_state=100
X = x5.reshape(-1, 40 , 99)
y = to_categorical(yli5) # yva2, yar2 ,ydo2 ,yli2
ytest = to_categorical(ytest_li) #[ytest_va,ytest_ar,ytest_do,ytest_li]
xtrain, ytrain = shuffle(X, y,random_state=random_state)


#########################################
xtrain = np.reshape(xtrain,[-1, 40, 99, 1])
xtest = np.reshape(xtest,[-1, 40, 99, 1])
tuner.search(xtrain, ytrain, epochs = 10, validation_data = (xtest, ytest) ,callbacks = [ClearTrainingOutput()])
# tuner.search(xtrain, ytrain, epochs = 10, validation_data = (xtest, ytest), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


Trial 30 Complete [00h 02m 35s]
val_accuracy: 0.7049999833106995

Best val_accuracy So Far: 0.7049999833106995
Total elapsed time: 00h 35m 03s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 224 and the optimal learning rate for the optimizer
is 0.001.



In [1]:
# # Build the model with the optimal hyperparameters and train it on the data and recheck it
# model = tuner.hypermodel.build(best_hps)
# model.fit(xtrain, ytrain,  epochs = 10, validation_data = (xtest, ytest))


In [ ]:
# # Generate generalization metrics
# scores = model.evaluate(xtest, ytest, verbose=1)#verbose =1 
# y_pred = model.predict(xtest)
# #     print("Fold number",fold_no)
# acc = scores[1]*100
# acc = round(acc, 4)
# print("Accuracy",acc)
# print("Loss",scores[0])
# print(classification_report(ytest.argmax(axis=1), y_pred.argmax(axis=1)))
# print(confusion_matrix(ytest.argmax(axis=1), y_pred.argmax(axis=1)))